In [1]:
import tensorflow as tf
import numpy as np
import uuid
import matplotlib.pyplot as plt

# np.random.seed(2018)

m = 2 # dimension
k_sq = 0.5
learning_rate = 0.01
epochs = 1000
batch_size = 100
x_stddev = 3
activation_fn_1 = tf.nn.sigmoid
activation_fn = tf.nn.relu
num_units_1 = 150
num_units_2 = 30
decay = 1 - 1e-10

test_averaging = 100
num_test_intervals = 10 # how many intervals to divide the test space into
num_test_points = num_test_intervals**m# how many test points across all spaces are evaluated

# Allow a batch of values to work
x0 = tf.placeholder(tf.float32, [None, m])
z = tf.placeholder(tf.float32, [None, m])

l1 = tf.layers.dense(
  x0, num_units_1, activation=activation_fn_1, use_bias=True)
l2 = tf.layers.dense(
  l1, m, activation=activation_fn_1, use_bias=True)

u1 = l2
u1_cost = (tf.norm(u1)**2) / batch_size

# The observed value for the second controller is the original controlled with noise
x1 = x0 + u1
y1 = x1 + z

l3 = tf.layers.dense(
  y1, num_units_2, activation=activation_fn, use_bias=True)
l4 = tf.layers.dense(
  l3, m, activation=activation_fn, use_bias=True)

u2 = -l4
x2 = x1 + u2

u2_cost = (tf.norm(x2) ** 2) / batch_size
wits_cost = (k_sq * u1_cost) + u2_cost

adaptive_learning_rate = tf.placeholder_with_default(learning_rate, [])

optimizer = tf.train.GradientDescentOptimizer(
  learning_rate=adaptive_learning_rate).minimize(wits_cost)

init_op = tf.global_variables_initializer()

tf.summary.scalar("WITS Cost", wits_cost)
merged_summary_op = tf.summary.merge_all()



INFO:tensorflow:Summary name WITS Cost is illegal; using WITS_Cost instead.


In [2]:
with tf.Session() as sess:
  sess.run(init_op)
  uniq_id = "/tmp/tensorboard-layers-api/" + uuid.uuid1().__str__()[:6]
  summary_writer = tf.summary.FileWriter(uniq_id, graph=tf.get_default_graph())
  x_train = np.random.normal(size=epochs * batch_size * m, scale=x_stddev)

  # Train for some epochs
  for step in range(epochs):
    x_batch = x_train[step: step + (batch_size * m)].reshape((batch_size, m))

    # Noise has variance 1
    z_batch = np.random.normal(size=(batch_size, m), scale=1)

    _, val, summary = sess.run(
      [optimizer, wits_cost, merged_summary_op],
      feed_dict={x0: x_batch, z: z_batch,
                 adaptive_learning_rate: learning_rate * (decay**step)})

    if step % 10 == 0:
      print("step: {}, value: {}".format(step, val))
      summary_writer.add_summary(summary, step)
  

  # Test over a continuous range of X
  x0_test=np.array(np.meshgrid
                   (np.linspace(-2*x_stddev, 2*x_stddev, num=num_test_intervals),
                    np.linspace(-2*x_stddev, 2*x_stddev, num=num_test_intervals)))
#   u1_test, u2_test, x1_test = np.zeros((m, num_test_points)), np.zeros((m, num_test_points)), np.zeros((m, num_test_points))
  u1_test = np.array(np.meshgrid(np.zeros(num_test_intervals), np.zeros(num_test_intervals)))
  u2_test = np.array(np.meshgrid(np.zeros(num_test_intervals), np.zeros(num_test_intervals)))
  x1_test = np.array(np.meshgrid(np.zeros(num_test_intervals), np.zeros(num_test_intervals)))

    
  for i in range(num_test_intervals):
    for j in range(num_test_intervals):
        u1t, u2t, x1t = 0, 0, 0
        for _ in range(test_averaging):
          x0_current_test = np.array([x0_test[0,i,j], x0_test[1,i,j]])
          u1tmp, u2tmp, x1tmp = sess.run(
                [u1, u2, x1], # return these variables
                feed_dict={x0: x0_current_test.reshape((1, 2)), z: np.random.normal(
                  size=(1, 2), scale=1)})

          u1t += u1tmp
          u2t += u2tmp
          x1t += x1tmp

        u1_test[0, i,j], u1_test[1,i,j] = u1t[0,0] / test_averaging, u1t[0,1]/ test_averaging
        u2_test[0, i,j], u2_test[1,i,j] = -u2t[0,0] / test_averaging, -u2t[0,1]/ test_averaging
        x1_test[0, i,j], x1_test[1,i,j] = x1t[0,0] / test_averaging, x1t[0,1]/ test_averaging

# l1, = plt.plot(x0_test, u1_test[0], label="U1 Test")
# l3, = plt.plot(x0_test, u2_test[0], label="U2 Test")
# plt.legend(handles=[l1, l3])
# plt.title("{} Unit NN With Activation Fn {}".format(
#   str(num_units_1), str(activation_fn)))
# # plt.savefig("figure_u_1.png")
# plt.show()

# plt.clf()
# l2, = plt.plot(x0_test, x1_test[0], label="X1 Test")
# plt.title("{} Unit NN With Activation Fn {}".format(
#   str(num_units_1), str(activation_fn)))
# plt.legend(handles=[l2])
# # plt.savefig("figure_x_1.png")
# plt.show()

step: 0, value: 19.887954711914062
step: 10, value: 12.185107231140137
step: 20, value: 11.642644882202148
step: 30, value: 11.573141098022461
step: 40, value: 11.366171836853027
step: 50, value: 11.644564628601074
step: 60, value: 11.017277717590332
step: 70, value: 10.490015983581543
step: 80, value: 10.313693046569824
step: 90, value: 10.222064018249512
step: 100, value: 9.732610702514648
step: 110, value: 9.16919231414795
step: 120, value: 9.688143730163574
step: 130, value: 10.044449806213379
step: 140, value: 9.466249465942383
step: 150, value: 9.241371154785156
step: 160, value: 9.372459411621094
step: 170, value: 9.46480941772461
step: 180, value: 9.476212501525879
step: 190, value: 9.452720642089844
step: 200, value: 9.46340274810791
step: 210, value: 9.028080940246582
step: 220, value: 9.380175590515137
step: 230, value: 9.671955108642578
step: 240, value: 9.15774154663086
step: 250, value: 8.83651351928711
step: 260, value: 8.752248764038086
step: 270, value: 9.3530149459838

In [3]:
u_test

NameError: name 'u0_test' is not defined